In [68]:
from IPython.core.debugger import set_trace

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot_ng as pydot


import sys
sys.path.append('../financial_utils/')

import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt

import performance as per

# DATASET

### Load and Compute Returns

In [69]:
price_table = pd.read_csv('history_files/SPY.csv')
vol_table = pd.read_csv('history_files/VIX.csv')

# Build Tick Table
tick_table = price_table[["Date","AdjClose"]]
tick_table.columns = ["Date", "Tick"]
# Get Return Table
returns = per.tick2ret(tick_table)
returns_table = pd.DataFrame({'Date':tick_table.Date[1:], 'Return': returns[:,0]})
returns_table = returns_table.set_index('Date')
# Drop columns of Vol Table
vol_table = vol_table[["Date", "Close"]]
vol_table.columns = ["Date", "Vol"]
vol_table.Vol = vol_table.Vol/100
vol_table = vol_table.set_index('Date')
# InnerJoin
retvol_table = pd.concat([returns_table,vol_table], join='inner', axis=1)


In [70]:
T_x = 5
n_fields = 2
T_y = 3
T_stride = 1

### Generate Dataset

In [71]:
def gen_dataset(T_x, T_y, n_fields, T_stride, retvol_table):
    """
    
    
    return
        x E (m, T_x)
        y E (m, T_y)
    """
    
    m = int(np.floor((returns.shape[0] - T_x) / T_stride))
    
    x = np.nan * np.ones((m-T_y*T_stride, T_x, n_fields))
    y = np.nan * np.ones((m-T_y*T_stride, T_y, n_fields))
    
    for i in range(m-T_y*T_stride):
        x[i,:,0] = retvol_table.Return[i*T_stride:i*T_stride+T_x].transpose()
        x[i,:,1] = retvol_table.Vol[i*T_stride:i*T_stride+T_x].transpose()
        y[i,:,0] = retvol_table.Return[i*T_stride+T_x:i*T_stride+T_x+T_y].transpose()
        y[i,:,1] = retvol_table.Vol[i*T_stride+T_x:i*T_stride+T_x+T_y].transpose()

    return x, y, m

In [72]:
def normalize_data(x, params=None):
    """
    
    return 
        x_norm
        x_norm_param = (mean, std)
    """
    if params == None:
        mean = np.mean(x, axis=(0,1)) 
        std = np.std(x, axis=(0,1))
    else:
        mean = params[0]
        std = params[1]
    x_norm_param = (mean, std)
    x_norm = (x - mean)/std
    
    return x_norm, x_norm_param

In [73]:
def denormalize_data(x_norm, x_norm_param):
    """"""
    
    mean = x_norm_param[0]
    std = x_norm_param[1]
    x = x_norm * std + mean
    
    return x

In [74]:
x, y, m = gen_dataset(T_x,T_y, n_fields, T_stride,retvol_table)
x, x_norm_param = normalize_data(x)
y, _ = normalize_data(y, x_norm_param)
print('Training Example: '+str(m))
print('X Shape: '+str(x.shape))
print('Y Shape: '+str(y.shape))

Training Example: 6090
X Shape: (6087, 5, 2)
Y Shape: (6087, 3, 2)


### Train / Val / Test

In [76]:
train_end = int(m * 0.7)

val_end = train_end + int(0.15 * m)

x_train = x[0:train_end]
x_val = x[train_end:val_end]
x_test = x[val_end:]

y_train = y[0:train_end]
y_val = y[train_end:val_end]
y_test = y[val_end:]

# Keras Model

### Encoder- Decoder

In [63]:
keras.backend.clear_session()

n_a = 100

encoder_LSTM = keras.layers.LSTM(units = n_a, return_state=True)
decoder_LSTM = keras.layers.LSTM(units = n_a, return_state=True, return_sequences=True)

flatter = keras.layers.Flatten()
dense = keras.layers.Dense(units = 100, activation='tanh')
relu_out = keras.layers.Dense(units = n_fields, activation='tanh')
concatenator = keras.layers.Lambda(lambda x: keras.backend.stack(x, axis=1))
last_slicor = keras.layers.Lambda(lambda x: x[:,-2:-1,:])

In [64]:
def define_model(T_x, T_y, n_fields, n_a):
    
    x_input = keras.layers.Input(shape=(T_x, n_fields))
    output_seq = list()
    
    _, encoder_h, encoder_c = encoder_LSTM(x_input)  
    
    decoder_input = keras.layers.Input(shape=(1,n_a))
    deco_input = decoder_input
    decoder_h = encoder_h
    decoder_c = encoder_c
    
    for _ in range(T_y):
        decoder_outputs, decoder_h, decoder_c = decoder_LSTM(deco_input, initial_state=[decoder_h, decoder_c])
        decoder_outputs_flat = flatter(decoder_outputs)
        out = dense(decoder_outputs_flat)
        out = relu_out(out)
        
        output_seq.append(out)
        deco_input = decoder_outputs
        
    output_seq = concatenator(output_seq)
    model = keras.models.Model(inputs=[x_input, decoder_input], outputs=output_seq)
    return model

In [65]:
model = define_model(T_x, T_y, n_fields, n_a)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 5, 2)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 100)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100), (None, 41200       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 1, 100), (No 80400       input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [66]:
optim = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optim, loss='mean_squared_error', metrics=['accuracy'])

In [67]:
decoder_in = np.zeros((x_train.shape[0],1,n_a))
model.fit(x=[x_train,decoder_in], y=y_train, shuffle=True, epochs = 100)

Epoch 1/100
4263/4263 [==============================] - 5s 1ms/step - loss: nan - acc: 1.0000
Epoch 2/100
4263/4263 [==============================] - 1s 338us/step - loss: nan - acc: 1.0000
Epoch 3/100
4263/4263 [==============================] - 1s 336us/step - loss: nan - acc: 1.0000
Epoch 4/100
4263/4263 [==============================] - 1s 337us/step - loss: nan - acc: 1.0000
Epoch 5/100
4263/4263 [==============================] - 1s 338us/step - loss: nan - acc: 1.0000
Epoch 6/100
4263/4263 [==============================] - 1s 338us/step - loss: nan - acc: 1.0000
Epoch 7/100
4263/4263 [==============================] - 1s 337us/step - loss: nan - acc: 1.0000
Epoch 8/100
4263/4263 [==============================] - 1s 346us/step - loss: nan - acc: 1.0000
Epoch 9/100
4263/4263 [==============================] - 1s 342us/step - loss: nan - acc: 1.0000
Epoch 10/100
4263/4263 [==============================] - 1s 339us/step - loss: nan - acc: 1.0000
Epoch 11/100
4263/4263 [=======

KeyboardInterrupt: 

In [30]:
decoder_in = np.zeros((x_test.shape[0],1,n_a))
model.evaluate(x=[x_test,decoder_in], y=y_test)

915/915 [==============================] - 0s 496us/step


[0.2564133430683548, 0.8147540988166475]

In [35]:
indexes = np.random.randint(0,x_test.shape[0], size=1)
x_in = x_test[indexes,:,:]
decoder_in = np.zeros((x_in.shape[0],1,n_a))
y_true = y_test[indexes,:,:]
y_pred = model.predict(x=[x_in,decoder_in])
y_pred_denorm = denormalize_data(y_pred, x_norm_param)
print('X:')
print(denormalize_data(x_in,x_norm_param))
print('Y:')
print(denormalize_data(y_true,x_norm_param))
print('LSTM Out:')
print(y_pred)
print('Y pred:')
print(y_pred_denorm)

X:
[[[0.0118108  0.2199    ]
  [0.00970978 0.1857    ]]]
Y:
[[[ 0.01981085  0.1608    ]
  [-0.00711086  0.17870001]]]
LSTM Out:
[[[-0.11139507 -0.07837891]
  [-0.0772531  -0.08006877]]]
Y pred:
[[[-0.00088692  0.19061824]
  [-0.00048694  0.19048061]]]


In [16]:
x_norm_param

(array([0.19700148]), array([0.0814408]))

## SNIPPETS

### Norm - Denorm Check

In [ ]:
a, b , m = gen_dataset(T_x,T_y,T_stride,retvol_table)
x, x_norm_param = normalize_data(a)
y, _ = normalize_data(b, x_norm_param)
x = denormalize_data(x, x_norm_param)
y = denormalize_data(y, x_norm_param)

plt.figure()
plt.hist((b-y).flatten(), 100)